In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Essential_Thrombocythemia"
cohort = "GSE57793"

# Input paths
in_trait_dir = "../../input/GEO/Essential_Thrombocythemia"
in_cohort_dir = "../../input/GEO/Essential_Thrombocythemia/GSE57793"

# Output paths
out_data_file = "../../output/preprocess/Essential_Thrombocythemia/GSE57793.csv"
out_gene_data_file = "../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE57793.csv"
out_clinical_data_file = "../../output/preprocess/Essential_Thrombocythemia/clinical_data/GSE57793.csv"
json_path = "../../output/preprocess/Essential_Thrombocythemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from patients with Essential Thrombocythemia (ET), Polycythemia Vera (PV), Primary Myelofibrosis (PMF)"
!Series_summary	"Microarrays were used to assess gene expression in patients with ET, PV, and PMF before and after treatment with IFNalpha2 in a paired design."
!Series_overall_design	"Total RNA was purified from whole blood and amplified to biotin-labeled aRNA and hybridized to microarray chips. Background correction, normalization, and gene expression index calculation were performed with the robust multi-array (rma) method. The regularized t-test limma for pairwise data was used to calculate differences in gene expression between patients before and after treatment with IFN-alpha2."
Sample Characteristics Dictionary:
{0: ['disease state: ET', 'disease state: PMF', 'disease state: PV'], 1: ['treatment: untreated', 'treatment: IFN-alpha2'], 2: ['tissue: Whole blood']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: In the sample characteristics dictionary, key 0 contains disease state information including ET
trait_row = 0
# Age and gender information are not available in the sample characteristics dictionary
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert the trait value to binary format:
    - 1 for Essential Thrombocythemia (ET)
    - 0 for other conditions (PMF, PV)
    """
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary based on ET status
    if value.upper() == 'ET':
        return 1
    elif value.upper() in ['PMF', 'PV']:
        return 0
    else:
        return None

def convert_age(value):
    """
    Placeholder function for age conversion (not used in this dataset)
    """
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """
    Placeholder function for gender conversion (not used in this dataset)
    """
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    else:
        return None

# 3. Save Metadata - Initial filtering validation
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features from the dataframe
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print(f"Preview of clinical features: {preview}")
    
    # Save the clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features: {'GSM1388566': [1.0], 'GSM1388567': [1.0], 'GSM1388568': [1.0], 'GSM1388569': [1.0], 'GSM1388570': [1.0], 'GSM1388571': [1.0], 'GSM1388572': [1.0], 'GSM1388573': [1.0], 'GSM1388574': [1.0], 'GSM1388575': [1.0], 'GSM1388576': [1.0], 'GSM1388577': [1.0], 'GSM1388578': [1.0], 'GSM1388579': [1.0], 'GSM1388580': [1.0], 'GSM1388581': [1.0], 'GSM1388582': [0.0], 'GSM1388583': [0.0], 'GSM1388584': [0.0], 'GSM1388585': [0.0], 'GSM1388586': [0.0], 'GSM1388587': [0.0], 'GSM1388588': [0.0], 'GSM1388589': [0.0], 'GSM1388590': [0.0], 'GSM1388591': [0.0], 'GSM1388592': [0.0], 'GSM1388593': [0.0], 'GSM1388594': [0.0], 'GSM1388595': [0.0], 'GSM1388596': [0.0], 'GSM1388597': [0.0], 'GSM1388598': [0.0], 'GSM1388599': [0.0], 'GSM1388600': [0.0], 'GSM1388601': [0.0], 'GSM1388602': [0.0], 'GSM1388603': [0.0], 'GSM1388604': [0.0], 'GSM1388605': [0.0], 'GSM1388606': [0.0], 'GSM1388607': [0.0], 'GSM1388608': [0.0], 'GSM1388609': [0.0], 'GSM1388610': [0.0], 'GSM1388611': [0.0], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examine the identifiers from the previous step
# These identifiers (like '1007_s_at', '1053_at') are typical Affymetrix probe IDs
# used on microarray platforms. They are not human gene symbols and need to be mapped.
# Affymetrix probe IDs typically follow this format with numbers and "_at" suffix.

# Based on my domain knowledge in biomedical research, these are clearly Affymetrix
# microarray probe IDs, not human gene symbols like BRCA1, TP53, etc.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on observation:
# - The gene expression data uses 'ID' as the key for identifiers (like '1007_s_at')
# - The gene annotation dataframe has both 'ID' column for probe IDs and 'Gene Symbol' column for gene symbols

# 2. Get gene mapping dataframe using the two columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# Print sample of the mapping to verify
print("Gene mapping sample (first 5 rows):")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Normalize gene symbols
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print the shape of the resulting gene expression data
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 gene symbols:")
print(gene_data.index[:5])


Gene mapping sample (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Gene expression data shape after mapping: (19845, 66)
First 5 gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Extracting clinical features...")
# Create the clinical features using the trait row identified in Step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from patients with Essential Thrombocythemia (ET), Polycythemia Vera (PV), and Primary Myelofibrosis (PMF)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (19845, 66)


Normalized gene data saved to ../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE57793.csv
Extracting clinical features...
Clinical data preview:
{'GSM1388566': [1.0], 'GSM1388567': [1.0], 'GSM1388568': [1.0], 'GSM1388569': [1.0], 'GSM1388570': [1.0], 'GSM1388571': [1.0], 'GSM1388572': [1.0], 'GSM1388573': [1.0], 'GSM1388574': [1.0], 'GSM1388575': [1.0], 'GSM1388576': [1.0], 'GSM1388577': [1.0], 'GSM1388578': [1.0], 'GSM1388579': [1.0], 'GSM1388580': [1.0], 'GSM1388581': [1.0], 'GSM1388582': [0.0], 'GSM1388583': [0.0], 'GSM1388584': [0.0], 'GSM1388585': [0.0], 'GSM1388586': [0.0], 'GSM1388587': [0.0], 'GSM1388588': [0.0], 'GSM1388589': [0.0], 'GSM1388590': [0.0], 'GSM1388591': [0.0], 'GSM1388592': [0.0], 'GSM1388593': [0.0], 'GSM1388594': [0.0], 'GSM1388595': [0.0], 'GSM1388596': [0.0], 'GSM1388597': [0.0], 'GSM1388598': [0.0], 'GSM1388599': [0.0], 'GSM1388600': [0.0], 'GSM1388601': [0.0], 'GSM1388602': [0.0], 'GSM1388603': [0.0], 'GSM1388604': [0.0], 'GSM1388605': [0.0], '

Linked data shape after handling missing values: (66, 19846)
Checking for bias in trait distribution...
For the feature 'Essential_Thrombocythemia', the least common label is '1.0' with 16 occurrences. This represents 24.24% of the dataset.
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Essential_Thrombocythemia/GSE57793.csv
